In [15]:
import face_recognition
import cv2
import numpy as np
import os

In [16]:
# Function to load known faces
def load_known_faces(directory):
    known_face_encodings = []
    known_face_names = []

    for file_name in os.listdir(directory):
        if file_name.endswith((".jpg", ".png")):
            image_path = os.path.join(directory, file_name)
            image = face_recognition.load_image_file(image_path)
            face_encodings = face_recognition.face_encodings(image)
            if face_encodings:
                known_face_encodings.append(face_encodings[0])
                known_face_names.append(os.path.splitext(file_name)[0])

    return known_face_encodings, known_face_names

In [17]:
# Function to calculate confidence from face distance function
def calculate_confidence(face_distance, face_match_threshold=0.7):
    if face_distance > face_match_threshold:
        linear_val = (1.0 - face_distance) / (0.1 - face_match_threshold)
        return max(0.0, min(1.0, linear_val)) * 100
    else:
        linear_val = (1.0 - face_distance) / (face_match_threshold - 0.1)
        return max(0.0, min(1.0, linear_val)) * 100

In [18]:
# Function to perform face recognition on a single frame
def recognize_faces(frame, known_face_encodings, known_face_names):
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    rgb_small_frame = small_frame[:, :, ::-1]
    
    # Find all the faces and face encodings in the current frame of video
    face_locations = face_recognition.face_locations(rgb_small_frame)
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

    face_names = []
    face_confidences = []
    for face_encoding in face_encodings:
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"
        confidence = 0.0

        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            confidence = calculate_confidence(face_distances[best_match_index])
            if confidence > 95:
                name = known_face_names[best_match_index]

        face_names.append(name)
        face_confidences.append(confidence)

    return face_locations, face_names, face_confidences

In [19]:
# Function to display the recognized faces on the frame
def display_results(frame, face_locations, face_names, face_confidences):
    for (top, right, bottom, left), name, confidence in zip(face_locations, face_names, face_confidences):
        # Scale back up face locations to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Only detect face with confidence > 95%
        if confidence > 95:
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            label = f"{name} ({confidence:.2f}%)"
            cv2.putText(frame, label, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
        else:
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            label = f"{name}"
            cv2.putText(frame, label, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    cv2.imshow('Video', frame)

In [20]:
def main():
    video_capture = cv2.VideoCapture(0)

    faces_dir = "faces"

    known_face_encodings, known_face_names = load_known_faces(faces_dir)

    process_this_frame = True

    while True:
        ret, frame = video_capture.read()

        if process_this_frame:
            face_locations, face_names, face_confidences = recognize_faces(frame, known_face_encodings, known_face_names)

        process_this_frame = not process_this_frame

        display_results(frame, face_locations, face_names, face_confidences)

        # Hit 'q' to quit!
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video_capture.release()
    cv2.destroyAllWindows()

In [29]:
if __name__ == "__main__":
    main()
